# ACCESS AND ENV SETUP

# <font style="color:red">→ Don't forget to connect VPNC</font>

SDumont manual (in Portuguese): <https://github.com/lncc-sered/manual-sdumont/wiki>

In [1]:
! ssh -TNf sd

In [2]:
%%bash
(
OPT="uid=1000,gid=1000,reconnect,cache=yes,kernel_cache,compression=no,ServerAliveCountMax=3"
SPWD=$(ssh -T sd "pwd")    # /prj
SSWD=/scratch${SPWD#/prj}  # /scratch
#sshfs sd:$SPWD  /prj      -o $OPT
#sshfs sd:$SSWD  /scratch  -o $OPT
sshfs sd:$SPWD  ~/prj      -o $OPT
sshfs sd:$SSWD  ~/scratch  -o $OPT
ssh -T sd << 'EOF'
source $HOME/conda/bin/activate base
PORT=18384
pkill -u $USER -f $PORT
if ss -tuln | grep -q ":$PORT"; then
    echo "Port $PORT is already in use."
else
    syncthing \
        --no-browser \
        --no-restart \
        --gui-address=0.0.0.0:$PORT \
        >/dev/null 2>&1 &
fi
PORT=18889
pkill -u $USER -f $PORT
if ss -tuln | grep -q ":$PORT"; then
    echo "Port $PORT is already in use."
else
    jupyter-lab \
        --no-browser \
        --notebook-dir=$HOME/ \
        --IdentityProvider.token='' \
        --ip=* \
        --port=$PORT \
        >/dev/null 2>&1 &
fi
sleep 2
hostname
ps -U $(id -u) -o pid,comm | grep -E "[j]upyter|[s]yncthing"
EOF
ssh -TNfL 8385:localhost:18384 sd    # syncthing
ssh -TNfL 8889:localhost:18889 sd    # jupyter
)

sdumont18
  83923 syncthing
  83924 jupyter-lab
  84045 syncthing
 858290 jupyter-lab
 859396 jupyter-lab
 863788 jupyter-lab
 864854 jupyter-lab
 865187 jupyter-lab
 865708 jupyter-lab
1240597 jupyter-lab
1243909 jupyter-lab
1246357 jupyter-lab
1467788 syncthing
1467801 syncthing
1467802 jupyter-lab
1477111 jupyter-lab


Now, you can connect to JL instance running on the SDumont login node. The `~/sdumont-sync/` directory is synchronized between the local machine and the SDumont.

- <http://localhost:8889>  JL (can takes minutes to go up)
- <http://localhost:8385>  ST

<hr style="height:10px;border-width:0;background-color:green">

# Shutdown

In [1]:
%%bash
(
sudo umount -f ~/prj ~/scratch
ssh -T sd << 'EOF'
pkill -u $USER -f jupyter
pkill -u $USER -f syncthing
hostname
EOF
ssh -O cancel -L 8889:localhost:18889 sd
ssh -O cancel -L 8385:localhost:18384 sd
ssh -O exit sd
)

sdumont18


Exit request sent.


# <font style="color:red">don't forget to disconnect VPNC</font>

<hr style="height:10px;border-width:0;background-color:green">

OLD - sdumont18:

In [13]:
%%bash
ssh -T sd << 'EOF1'
ssh -T sdumont18 << 'EOF2'
pkill -u ${USER} -f jupyter
pkill -u ${USER} -f syncthing
hostname
EOF2
EOF1

ssh -NfTL 8890:sdumont18:18890 sd

sdumont18


OLD - sdumont13,14:

In [14]:
%%bash
ssh -T sd << 'EOF1'
pkill -u ${USER} -f jupyter
pkill -u ${USER} -f syncthing
hostname
ssh -T sdumont13 << 'EOF2'
pkill -u ${USER} -f jupyter
pkill -u ${USER} -f syncthing
hostname
EOF2

ssh -T sdumont14 << 'EOF2'
pkill -u ${USER} -f jupyter
pkill -u ${USER} -f syncthing
hostname
EOF2
EOF1


sdumont14
sdumont13
sdumont14


máquina local:

<hr style="height:10px;border-width:0;background-color:blue">

# When the connection drops (vpnc):

In [ ]:
%%bash
ssh -O exit sd
ssh -NfT sd
ssh -NfTL 8890:sdumont18:18890 sd
ssh -NfTL 8891:sdumont18:18891 sd
ssh -T sd hostname

- "ssh -O exit sd" (exit) and then "ssh -Nf sd" (enter)
- disconnects everything and connects again (JL returns)
- if you connect to another login node, the previous session that was on the previous node remains there.
- if you need to kill a process, enter it with ssh XXXX and pkill -u ${USER} -f jupyter

---

SSH

- -f : Requests ssh to go to background. The recommended way to start X11 programs at a remote site (does not work if there is 'EOF')
- -N : Do not execute a remote command. This is useful for just forwarding ports
- -T : Disable pseudo-tty allocation. (tty = to have interactive prompt)
- -L : Port forwarding
- (the sequence and order of the flags is important)

    vi ~/.ssh/config
    Host sd
      HostName login.sdumont.lncc.br   (or 146.134.143.249)
      User xxxx
      ControlMaster auto
      ControlPath ~/.ssh/remote_sd

OLD - sdumont18 - conda in the /prj directory

In [4]:
%%bash
ssh -T sd << 'EOF1'
ssh -T sdumont18 << 'EOF2'
module load sequana/current
module load sdbase
#module load cuda/10.0_sequana
#module load cuda/10.0
#module load cudnn/7.6_cuda-10.0_sequana
#module load cudnn/7.6_cuda-10.0
#module load gcc/7.4_sequana
#module load gcc/7.4
#module load cmake/3.23.2_sequana
#module load cmake/3.23.2
#module load openmpi/gnu/4.0.1_sequana
pkill -u ${USER} -f 18890
module load gcc/11.1 netcdf/4.9.2_openmpi-4.1.6_gnu openblas/0.3.19_gnu
source $HOME/conda/bin/activate base
jupyter-lab --no-browser --LabApp.token=x --ip=0.0.0.0 --port=18890 &>/dev/null&
hostname && sleep 2 && ps x | grep "[j]upyter\|[s]yncthing"
EOF2
EOF1

ssh -NfTL 8890:sdumont18:18890 sd

Loading SEQUANA Software environment
Loading SDumont BASE Software environment


sdumont18
  9478 ?        D      0:00 /prj/ampemi/eduardo.miranda2/conda/bin/python3.10 /prj/ampemi/eduardo.miranda2/conda/bin/jupyter-lab --no-browser --LabApp.token=x --ip=0.0.0.0 --port=18890
193043 ?        Sl     0:03 /prj/ampemi/eduardo.miranda2/conda/bin/python3.10 /prj/ampemi/eduardo.miranda2/conda/bin/jupyter-lab --no-browser --LabApp.token=x --ip=0.0.0.0 --port=18890


- <http://localhost:8890/lab?token=x> (may take a long time to upload, 1 min. or more)
- only works on the machine running ssh
- programs have a 5 min. limit running on the login node
- para 20 min: salloc --partition=sequana_gpu_dev --nodes=1 --ntasks=1 --job-name test01
- salloc --partition=sequana_gpu_shared --nodes=1 --ntasks=1 --job-name test01

---

OLD - sdumont18 - miniconda int the /scratch  directory

In [5]:
%%bash
ssh -T sd << 'EOF1'
ssh -T sdumont18 << 'EOF2'
module purge
module load sequana/current
module load sdbase
module load cuda/10.0_sequana
module load cudnn/7.6_cuda-10.0_sequana
module load openmpi/gnu/4.0.1_sequana
module load gcc/7.4_sequana
module load cmake/3.23.2_sequana
source /scratch${HOME#/prj}/miniconda3/bin/activate base
pkill -u ${USER} -f 18891
jupyter-lab --no-browser --LabApp.token='' --ip=0.0.0.0 --port=18891 &>/dev/null&
hostname && sleep 2 && ps x | grep [j]upyter
EOF2
EOF1

ssh -NfTL 8891:sdumont18:18891 sd

Loading SEQUANA Software environment
Loading SDumont BASE Software environment
ModuleCmd_Load.c(213):ERROR:105: Unable to locate a modulefile for 'cuda/10.0_sequana'
ModuleCmd_Load.c(213):ERROR:105: Unable to locate a modulefile for 'cudnn/7.6_cuda-10.0_sequana'
ModuleCmd_Load.c(213):ERROR:105: Unable to locate a modulefile for 'openmpi/gnu/4.0.1_sequana'
ModuleCmd_Load.c(213):ERROR:105: Unable to locate a modulefile for 'gcc/7.4_sequana'
ModuleCmd_Load.c(213):ERROR:105: Unable to locate a modulefile for 'cmake/3.23.2_sequana'


sdumont18
 99058 ?        Sl     0:05 /prj/ampemi/eduardo.miranda2/conda/bin/python3.10 /prj/ampemi/eduardo.miranda2/conda/bin/jupyter-lab --no-browser --LabApp.token=x --ip=0.0.0.0 --port=18890
 99599 ?        Ssl    0:00 /prj/ampemi/eduardo.miranda2/conda/bin/python -m ipykernel_launcher -f /prj/ampemi/eduardo.miranda2/.local/share/jupyter/runtime/kernel-2cd5f5a5-6b51-4144-b781-f73bba7ed221.json
 99606 ?        Ssl    0:00 /prj/ampemi/eduardo.miranda2/conda/bin/python -m ipykernel_launcher -f /prj/ampemi/eduardo.miranda2/.local/share/jupyter/runtime/kernel-c4f96dd9-520e-4b47-bf4d-b82a12258ce2.json
101622 ?        S      0:00 /scratch/ampemi/eduardo.miranda2/miniconda3/bin/python3.11 /scratch/ampemi/eduardo.miranda2/miniconda3/bin/jupyter-lab --no-browser --LabApp.token=x --ip=0.0.0.0 --port=18891


- <http://localhost:8891>